# 1. 키워드 추출

## 1-1 제목에서 추출

In [1]:
# 불필요한 경고 출력을 방지
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split


In [3]:
# '-' 깨짐 해결 모듈
import matplotlib.pylab as plt

import matplotlib as mpl

plt.rcParams["font.family"] = "Malgun Gothic"  # For Windows
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (8,4)
print(plt.rcParams["font.family"])

# 마이너스 깨짐 해결
mpl.rcParams["axes.unicode_minus"] = False

['Malgun Gothic']


### 데이터 읽기

In [4]:
# techworld
df_bloter = pd.read_csv('../data_crawling_D/before_keywords/news_recentit_data_202306231600.csv')
print(df_bloter.shape)

(335, 9)


In [5]:
df_bloter.head()


,news_date,news_title,news_text_sm,url_in,news_writer,thumb_addr,tags_string,news_key,news_site
0,2023.04.05,개발자 ChatGPT로 보안 정복하기: 침해 사고 분석,"정보통신망 제2조 제1항 제7호에 따르면, “침해 사고는 해킹, 컴퓨터 바이러스, ...",https://yozm.wishket.com/magazine/detail/1960/,김보안,https://yozm.wishket.com/media/news/1960/hacke...,NaN,침해,요즘IT
1,2023.04.04.,코드 리뷰가 개발 문화에 미치는 영향,요즘 매일 보는 앱 중에 깃허브(github)가 있습니다. 개발에서 손을 뗀 지 벌...,https://yozm.wishket.com/magazine/detail/1957/,안영회,https://yozm.wishket.com/media/news/1957/team-...,NaN,면접,요즘IT
2,2023.04.03.,주니어 개발자를 위한 엄청 쉬운 TCP/IP 4계층 이야기,이번 글에서는 이전 글 “택배 배송 과정으로 알아보는 엄청 쉬운 컴퓨터 네트워크”를...,https://yozm.wishket.com/magazine/detail/1956/,Morian,https://yozm.wishket.com/media/news/1956/엄청쉬운섬...,NaN,Git,요즘IT
3,2023.03.30.,쿠버네티스 환경의 대표적 보안 기능 3가지,쿠버네티스 환경에서의 보안이란 쿠버네티스 클러스터의 안정성과 보안성을 유지하는 것을...,https://yozm.wishket.com/magazine/detail/1953/,테크유람,https://yozm.wishket.com/media/news/author/%EA...,NaN,쿠버네티스,요즘IT
4,2023.03.28.,"ChatGPT 플러그인, 그 의미와 의도는 무엇?",2023년 3월은 아마 AI역사에 꽤나 중요한 기간이 될 듯합니다. ChatGPT용...,https://yozm.wishket.com/magazine/detail/1951/,김영욱,https://yozm.wishket.com/media/news/author/%EA...,NaN,주니어,요즘IT


In [6]:
df_bloter.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   news_date     335 non-null    object 
 1   news_title    335 non-null    object 
 2   news_text_sm  335 non-null    object 
 3   url_in        335 non-null    object 
 4   news_writer   335 non-null    object 
 5   thumb_addr    335 non-null    object 
 6   tags_string   0 non-null      float64
 7   news_key      335 non-null    object 
 8   news_site     335 non-null    object 
dtypes: float64(1), object(8)
memory usage: 23.7+ KB


In [7]:
# df_techworld.columns
# idx 떼내기
df_bloter = df_bloter[[ 'news_date', 'news_title', 'news_text_sm', 'url_in',
       'news_writer', 'tags_string', 'thumb_addr', 'news_site']]

In [8]:
# df_recentit news_date 수정하기 (맨우측에 . 삭제)
df_bloter['news_date']= df_bloter['news_date'].str.replace(r'\.', '-', regex=True)
df_bloter['news_date']

# [ 출처: https://seong6496.tistory.com/ ]

0       2023-04-05
1      2023-04-04-
2      2023-04-03-
3      2023-03-30-
4      2023-03-28-
          ...     
330    2022-06-14-
331    2022-06-13-
332    2022-06-09-
333    2022-06-03-
334    2022-06-03-
Name: news_date, Length: 335, dtype: object

In [9]:
# 결측값 채우기
df_bloter.fillna('', inplace=True)
# df_itworld.fillna('')

## KoNLPy로 단어 단위 분해

In [10]:
# datetime(날짜), os 호출

import datetime
import os
# 불용.txt 읽기 위해 csv 파일 호출
import csv
import re
# konlpy 호출
from konlpy.tag import Okt

# 자연어, 불용어 처리 및 wordcloud 생성을 위해
# 집계 메소드
from collections import Counter
from konlpy.tag import Komoran
# https://datascienceschool.net/03%20machine%20learning/03.01.01%20NLTK%20%EC%9E%90%EC%97%B0%EC%96%B4%20%EC%B2%98%EB%A6%AC%20%ED%8C%A8%ED%82%A4%EC%A7%80.html
from nltk.tokenize import word_tokenize


In [11]:
# 모듈화
def get_title_keywords(df):
    # 컬럼 'news_title' 가져오기 
    rdr = df['news_title'] 
    
    # Okt 객체 선언
    okt = Okt()
    ###  불용어 빼기
    # 불용어 파일 읽기
    stop_words = []
    with open('../data_analysis_test/불용.txt', 'r', encoding='utf-8') as file:
       stop_words = file.read().splitlines()
    
    h_word_list = []
    for w_bundle in rdr:
      # print(okt.nouns(w_bundle))
        list = okt.nouns(w_bundle)
        # 특수문자 제거
        tokens = ",".join(list)
        tokens = re.sub('[\{\}\[\]\/?.;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', tokens)
        list = tokens.split(",")
        # 한글자인 명사, ',' 빼기 (예외:웹,앱,폰,챗)
        for i,v in enumerate(list):
            if len(v) < 2 and (v != '웹' and v != '앱' and v != '폰' and v != '챗'):
                list.pop(i)
            elif v == ',':
                list.pop(i)
            # 불용어 제거
            elif i in stop_words:
                list.pop(i)
        list_str = ",".join(list)
       
        h_word_list.append(list_str)
    return h_word_list
    # print(h_word_list)
    # len(techworld_h_word_list)

bloter_h_word_list = get_title_keywords(df_bloter)

## 해당 리스트 목록을 새로운 컬럼으로 추가

In [12]:
df_bloter['news_key'] = bloter_h_word_list
df_bloter['news_key']

0          개발자,보안,정복,침해,사고,분석
1              코드,리뷰,개발,문화,영향
2              주니어,개발자,계층,이야기
3        쿠버,티스,환경,대표,보안,기능,가지
4               플러그인,의미,의도,무엇
                ...          
330          웹사이트,슬랙,메타,태그,가지
331    여러분,나은,플러터,개발자,가지,프로젝트
332              가독성,다섯,가지,습관
333         프론트엔드,개발자,알고리즘,공부
334                   클라우드,시작
Name: news_key, Length: 335, dtype: object

## 1-2 news_text_sm에서 키워드 추출하기

In [13]:
# 모듈화
def get_text_keywords(df):
    # 컬럼 'news_title' 가져오기 
    rdr = df['news_text_sm'] 
    
    # Okt 객체 선언
    okt = Okt()
    ###  불용어 빼기
    # 불용어 파일 읽기
    stop_words = []
    with open('../data_analysis_test/불용.txt', 'r', encoding='utf-8') as file:
       stop_words = file.read().splitlines()
    
    h_word_list = []
    for w_bundle in rdr:
      # print(okt.nouns(w_bundle))
        list = okt.nouns(w_bundle)
        # 특수문자 제거
        tokens = ",".join(list)
        tokens = re.sub('[\{\}\[\]\/?.;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', tokens)
        list = tokens.split(",")
        # 한글자인 명사, ',' 빼기 (예외:웹,앱,폰,챗)
        for i,v in enumerate(list):
            if len(v) < 2 and (v != '웹' and v != '앱' and v != '폰' and v != '챗'):
                list.pop(i)
            elif v == ',':
                list.pop(i)
            # 불용어 제거
            elif i in stop_words:
                list.pop(i)
        list_str = ",".join(list)
       
        h_word_list.append(list_str)
    return h_word_list
    # print(h_word_list)
    # len(techworld_h_word_list)

bloter_txt_word_list = get_title_keywords(df_bloter)


In [14]:
df_bloter['news_txt_key'] = bloter_txt_word_list
df_bloter['news_txt_key']

0          개발자,보안,정복,침해,사고,분석
1              코드,리뷰,개발,문화,영향
2              주니어,개발자,계층,이야기
3        쿠버,티스,환경,대표,보안,기능,가지
4               플러그인,의미,의도,무엇
                ...          
330          웹사이트,슬랙,메타,태그,가지
331    여러분,나은,플러터,개발자,가지,프로젝트
332              가독성,다섯,가지,습관
333         프론트엔드,개발자,알고리즘,공부
334                   클라우드,시작
Name: news_txt_key, Length: 335, dtype: object

In [15]:
# df_techworld

### 'news_key', 'news_txt_key' 값 합치기

In [16]:
df_bloter['total_keywords'] = df_bloter[['news_key', 'news_txt_key']].apply(','.join, axis=1)
print(df_bloter)

       news_date                            news_title  \
0     2023-04-05        개발자 ChatGPT로 보안 정복하기: 침해 사고 분석   
1    2023-04-04-                  코드 리뷰가 개발 문화에 미치는 영향   
2    2023-04-03-      주니어 개발자를 위한 엄청 쉬운 TCP/IP 4계층 이야기   
3    2023-03-30-               쿠버네티스 환경의 대표적 보안 기능 3가지   
4    2023-03-28-           ChatGPT 플러그인, 그 의미와 의도는 무엇?   
..           ...                                   ...   
330  2022-06-14-        내 웹사이트를 슬랙에서 돋보이게 해줄 메타 태그 8가지   
331  2022-06-13-      여러분을 더 나은 플러터 개발자로 만들어줄 7가지 프로젝트   
332  2022-06-09-             SQL 가독성을 높이는 다섯 가지 사소한 습관   
333  2022-06-03-           프론트엔드 개발자에게도 알고리즘 공부가 중요할까?   
334  2022-06-03-  AWS로 클라우드 시작하기: ⑤IAM & Organizations   

                                          news_text_sm  \
0    정보통신망 제2조 제1항 제7호에 따르면, “침해 사고는 해킹, 컴퓨터 바이러스, ...   
1    요즘 매일 보는 앱 중에 깃허브(github)가 있습니다. 개발에서 손을 뗀 지 벌...   
2    이번 글에서는 이전 글 “택배 배송 과정으로 알아보는 엄청 쉬운 컴퓨터 네트워크”를...   
3    쿠버네티스 환경에서의 보안이란 쿠버네티스 클러스터의 안정성과 보안성을 유지하는 것을...   
4    2023년 3월

### 중복 키워드 처리

In [17]:
# from collections import OrderedDict

# list = '티맥스,티베로,인텔,코리아,비즈니스,연속성,혁신,전략,웨비,개최,티맥스,티베로,인텔,코리아,비즈니스,연속성,혁신,전략,웨비,개최'
# list_sep = list.split(",")
# list_sep

# a = " ".join(OrderedDict.fromkeys(list_sep))
# print(a)




In [18]:
from collections import OrderedDict

t_keyword_list = []

for keys in df_bloter['total_keywords']:
    # print([keys]) # 일단 전부 ,로 split()해줘야 단어끼리로 분리됨
    keys_list = keys.split(",")
    key_list_str = ','.join(OrderedDict.fromkeys(keys_list))
    # print(key_list_str)
    # 2차원 리스트 만들기
    t_keyword_list.append(key_list_str)

t_keyword_list


['개발자,보안,정복,침해,사고,분석',
 '코드,리뷰,개발,문화,영향',
 '주니어,개발자,계층,이야기',
 '쿠버,티스,환경,대표,보안,기능,가지',
 '플러그인,의미,의도,무엇',
 '컴퓨팅,사고,개발,실력,공부',
 '큐어,코딩,보안,이슈,코드',
 '개발자,검색,엔진,파인드,활용',
 '프롬,프트,엔지니어링,프롬프트,인젝션',
 '콘텐츠,웹,재생',
 '아두,만사,머신,러닝,가지,알고리즘',
 '코드,재사용,결정,고려',
 '결합,원칙,활용,소프트웨어,설계',
 '개발자,거듭,나기,가지,기본,소양,기억',
 '게임,금융,의료,비즈니스,곳곳,이상,탐지,모델',
 '쿠버,티스,로드,통신,방법,인그레스',
 '면접,개발자,지망',
 '웹,소켓,개발,것',
 '쿠버,티스,로드,통신,방법,서비스',
 '주니어,서버,기초,상식',
 '택배,배송,과정,컴퓨터,네트워크',
 '코드,리뷰,스포츠',
 '개발자,활용',
 '게이트웨이,보기,스로틀링,알고리즘,구현',
 '개발자,가지,특징',
 '주니어,개발자,추천,도서',
 '우리,월,패드,사생활,노출',
 '소프트웨어,설계,깨달,조건',
 '코딩,문서,작성,콰르토,보기',
 '주니어,개발자,네트워크,이야기',
 '프로그래밍,종말',
 '개발자,영어,공부,방법',
 '워드,보안,사고,국가,안보,금융,분야',
 '웹,위해,이해,강제,기술',
 '개발자,컨퍼런스',
 '워드,보안,사고,공통,기술,분야',
 '웹,위해,이해,작동,원리',
 '웹,호환성,문제,버전,브라우저,지원',
 '주니어,개발자,퇴사,공부',
 '자바스크립트,작성,습관,작별',
 '회고,개발,트렌드,미국,대학,석사',
 '큐어,코딩,의미,실천,방안,점검,유형',
 '다음,가지,언어',
 '파이썬,스트림,데이터,대시보드,만들기',
 '큐어,코딩,의미,실천,방안',
 '최고,코드,개발,도구',
 '놀리,마이크로,서비스,아키텍처,선택',
 '여기,보기,프로세스,마음,루트,결심',
 '여기,보기,이,자유,위해,실행,권한,점검',
 '구축,멀티,패스,개발,환경',


In [19]:
df_bloter['total_keywords'] = t_keyword_list
df_bloter['total_keywords'] 

0          개발자,보안,정복,침해,사고,분석
1              코드,리뷰,개발,문화,영향
2              주니어,개발자,계층,이야기
3        쿠버,티스,환경,대표,보안,기능,가지
4               플러그인,의미,의도,무엇
                ...          
330          웹사이트,슬랙,메타,태그,가지
331    여러분,나은,플러터,개발자,가지,프로젝트
332              가독성,다섯,가지,습관
333         프론트엔드,개발자,알고리즘,공부
334                   클라우드,시작
Name: total_keywords, Length: 335, dtype: object

### 'news_key', 'news_text_key' 컬럼 삭제

In [20]:
df_bloter.drop(['news_key', 'news_txt_key'], axis=1, inplace=True)

In [21]:
# df_techworld

## df_cwn csv 파일에 반영


In [22]:
df_bloter.to_csv("../data_crawling_D/after_keywords/news_recentit_k_t.csv", mode='w')

# 2. 신뢰도 평가 : news_titlw, news_text_sm의 데이터를 KRWORDRANK로 추출

In [23]:
import pandas as pd
import numpy as np

In [24]:
# 먼저 news_text_sm 컬럼의 값들을 뭉쳐서 하나의 string(각 요소는 '\n'로 나누기)
# 으로 통합
import csv # sql_cwn_data에서 갖고 오기

f = open("../data_crawling_D/after_keywords/news_recentit_k_t.csv", "r", encoding='utf8')
reader = csv.reader(f)
next(reader)
# print(reader)
data = []
for row in reader:
    data.append(row)
    # print(row)
f.close()



In [25]:
# data


In [26]:
# dataframe으로 만들기
df_bloter = pd.DataFrame(data=data, columns=['idx','news_date', 'news_title', 'news_text_sm', 'url_in', 'news_writer', 'tags_string', 'thumb_addr', 'news_site', 'total_keywords'])  
# idx (인덱스) 제거
df_bloter = df_bloter[['news_date', 'news_title', 'news_text_sm', 'url_in', 'news_writer', 'tags_string', 'thumb_addr', 'news_site','total_keywords']]
df_bloter

,news_date,news_title,news_text_sm,url_in,news_writer,tags_string,thumb_addr,news_site,total_keywords
0,2023-04-05,개발자 ChatGPT로 보안 정복하기: 침해 사고 분석,"정보통신망 제2조 제1항 제7호에 따르면, “침해 사고는 해킹, 컴퓨터 바이러스, ...",https://yozm.wishket.com/magazine/detail/1960/,김보안,,https://yozm.wishket.com/media/news/1960/hacke...,요즘IT,"개발자,보안,정복,침해,사고,분석"
1,2023-04-04-,코드 리뷰가 개발 문화에 미치는 영향,요즘 매일 보는 앱 중에 깃허브(github)가 있습니다. 개발에서 손을 뗀 지 벌...,https://yozm.wishket.com/magazine/detail/1957/,안영회,,https://yozm.wishket.com/media/news/1957/team-...,요즘IT,"코드,리뷰,개발,문화,영향"
2,2023-04-03-,주니어 개발자를 위한 엄청 쉬운 TCP/IP 4계층 이야기,이번 글에서는 이전 글 “택배 배송 과정으로 알아보는 엄청 쉬운 컴퓨터 네트워크”를...,https://yozm.wishket.com/magazine/detail/1956/,Morian,,https://yozm.wishket.com/media/news/1956/엄청쉬운섬...,요즘IT,"주니어,개발자,계층,이야기"
3,2023-03-30-,쿠버네티스 환경의 대표적 보안 기능 3가지,쿠버네티스 환경에서의 보안이란 쿠버네티스 클러스터의 안정성과 보안성을 유지하는 것을...,https://yozm.wishket.com/magazine/detail/1953/,테크유람,,https://yozm.wishket.com/media/news/author/%EA...,요즘IT,"쿠버,티스,환경,대표,보안,기능,가지"
4,2023-03-28-,"ChatGPT 플러그인, 그 의미와 의도는 무엇?",2023년 3월은 아마 AI역사에 꽤나 중요한 기간이 될 듯합니다. ChatGPT용...,https://yozm.wishket.com/magazine/detail/1951/,김영욱,,https://yozm.wishket.com/media/news/author/%EA...,요즘IT,"플러그인,의미,의도,무엇"
...,...,...,...,...,...,...,...,...,...
330,2022-06-14-,내 웹사이트를 슬랙에서 돋보이게 해줄 메타 태그 8가지,"링크를 입력했을 때 나오는 제목과 부제목, 썸네일은 웹사이트의 첫인상을 결정하는 요...",https://yozm.wishket.com/magazine/detail/1526/,재그지그의 개발 블로그,,https://yozm.wishket.com/media/news/1526/image...,요즘IT,"웹사이트,슬랙,메타,태그,가지"
331,2022-06-13-,여러분을 더 나은 플러터 개발자로 만들어줄 7가지 프로젝트,본문은 요즘IT와 번역가 윌리(Willy)가 함께 만든 해외 번역 콘텐츠입니다. 이...,https://yozm.wishket.com/magazine/detail/1525/,요즘IT의 번역글들,,https://yozm.wishket.com/media/news/1525/image...,요즘IT,"여러분,나은,플러터,개발자,가지,프로젝트"
332,2022-06-09-,SQL 가독성을 높이는 다섯 가지 사소한 습관,지독하게 읽기 힘든 SQL 문을 해석해 본 적 있으신가요? 마치 암호를 읽는 것처럼...,https://yozm.wishket.com/magazine/detail/1519/,데이터리안,,https://yozm.wishket.com/media/news/1519/제목을-입...,요즘IT,"가독성,다섯,가지,습관"
333,2022-06-03-,프론트엔드 개발자에게도 알고리즘 공부가 중요할까?,우리는 왜 알고리즘을 공부할까요? 기초 소양을 쌓기 위해서? 좀 더 나은 개발자가 ...,https://yozm.wishket.com/magazine/detail/1513/,재그지그의 개발 블로그,,https://yozm.wishket.com/media/news/1513/image...,요즘IT,"프론트엔드,개발자,알고리즘,공부"


## news_title과 news_text_sm를 합친 컬럼 생성하기

In [27]:
df_bloter['title_n_text'] = df_bloter[['news_title', 'news_text_sm']].apply(','.join, axis=1)
# df_techworld['title_n_text']


In [28]:
# 제목과 내용을 합친 str 컬럼을 news_date와 같이 엮어 df화
df_bloter_dnt = df_bloter[['news_date', 'title_n_text']]
# csv 파일로 저장하기
df_bloter_dnt.to_csv("../data_analysis_test/test_data/recentit_dnt.txt", mode='w', header=False, index=False, sep='\t')

In [29]:
# df_cwn_date = df_cwn['news_date'].to_list()
# df_cwn_text = df_cwn['news_text_sm'].to_list()
# df_cwn_date_n_text = {}
# for date, text in zip(df_cwn_date, df_cwn_text):
#     df_cwn_date_n_text[date] = text
                          
# print('dict zip : ', df_cwn_date_n_text)

# df_cwn_text

In [30]:
df_bloter_dnt

,news_date,title_n_text
0,2023-04-05,"개발자 ChatGPT로 보안 정복하기: 침해 사고 분석,정보통신망 제2조 제1항 제..."
1,2023-04-04-,"코드 리뷰가 개발 문화에 미치는 영향,요즘 매일 보는 앱 중에 깃허브(github)..."
2,2023-04-03-,"주니어 개발자를 위한 엄청 쉬운 TCP/IP 4계층 이야기,이번 글에서는 이전 글 ..."
3,2023-03-30-,"쿠버네티스 환경의 대표적 보안 기능 3가지,쿠버네티스 환경에서의 보안이란 쿠버네티스..."
4,2023-03-28-,"ChatGPT 플러그인, 그 의미와 의도는 무엇?,2023년 3월은 아마 AI역사에..."
...,...,...
330,2022-06-14-,"내 웹사이트를 슬랙에서 돋보이게 해줄 메타 태그 8가지,링크를 입력했을 때 나오는 ..."
331,2022-06-13-,"여러분을 더 나은 플러터 개발자로 만들어줄 7가지 프로젝트,본문은 요즘IT와 번역가..."
332,2022-06-09-,"SQL 가독성을 높이는 다섯 가지 사소한 습관,지독하게 읽기 힘든 SQL 문을 해석..."
333,2022-06-03-,"프론트엔드 개발자에게도 알고리즘 공부가 중요할까?,우리는 왜 알고리즘을 공부할까요?..."


In [31]:
# f = open("../know_arc_pjt/data_crawling_D/data_sql_total_edited/total_news_t_202306221233.csv", "r", encoding='utf8')
# reader = csv.reader(f)

# for row in reader:
#     data = list(reader)
# # print(data)
# f.close()
# # dataframe으로 만들기
# df_total_t = pd.DataFrame(data=data, columns=['news_date', 'news_title', 'news_text_sm', 'url_in', 'news_writer', 'tags_string', 'thumb_addr', 'news_site'])  
# df_total_t

In [32]:
# 날짜당 제목에서 추출한 키워드 목록
def get_date_text(fname):
    # 튜토리얼에서 이용하는 `fname` 파일은 영화평과 평점이 \t 으로 구분된 two column tsv 파일입니다.
    # 예시는 이 cell 의 output 을 참고하세요.
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f] 
        docs = [doc for doc in docs if len(doc) == 2]

        if not docs:
            return [], []
        
        dates, texts = zip(*docs)
        return dates, texts

# cwn.kr data (df_cwn_date_n_text)
fname = '../data_analysis_test/test_data/cwn_dnt.txt'
# 이걸로 
dates, texts = get_date_text(fname)
# dates = list(dates)
# texts = list(texts)
# 위쪽의 5개 출력하기
# with open(fname, encoding="utf-8") as f:
#     for _ in range(5):
#         print(next(f).strip())

In [33]:
# dates
# texts

In [34]:
# KR-WordRank에 필요한 라이브러리
import sys
import re
sys.path.append('../')
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
import krwordrank
# print(krwordrank.__version__)

단어 추출에 영어/숫자를 포함할 예정이라면 normalize함수를 이용하여 텍스트를 normalize할 것

In [35]:
with open('../data_analysis_test/test_data/recentit_dnt_norm.txt', 'w', encoding='utf-8') as f:
    for date, text in zip(dates, texts):
        text = normalize(text, english=True, number=True)
        # 특수문자는 제거
        text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", text)
        f.write('%s\t%s\n' % (date, text))

## 해당 데이터에서 핵심 키워드들을 뽑아보자

In [36]:
# df_cwn_date_n_text
fname = '../data_analysis_test/test_data/recentit_dnt_norm.txt'
dates_n, texts_n = get_date_text(fname)
# 여기까진 dates_n, texts_n이 서로 분리되어있음

In [37]:
wordrank_extractor = KRWordRank(
    min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

# 여기서 for문
# text_key_list = []
# for text in texts_n:
keywords, rank, graph = wordrank_extractor.extract(texts_n, beta, max_iter)
    

scan vocabs ... 
num vocabs = 3683
done = 10


In [38]:
# keywords

위와 같이 vocabulary를 미리 설정하거나 decaying factor를 단어별로 다르게 (bias) 할당할 수 있으며, 모든 단어의 랭킹의 총 합은 vocabulary size와 같음. 즉 default decaying factor는 1.0

In [39]:
# 불용어 파일 읽기
stop_words = []
with open('../data_analysis_test/불용.txt', 'r', encoding='utf-8') as file:
   stop_words = file.read().splitlines()
# keyword_list
# # 불용어 제거
pop_list = []
for keyword in keywords:
    # print(keyword)
    # print(type(keyword))
    if keyword in stop_words:
        pop_list.append(keyword)
        # print(keyword)
        
# print(pop_list)
for p in pop_list:
    # print(p)
    keywords.pop(p, None)

# # filtered_tokens =  [token for token in keyword_list[0] if token not in stop_words]
# # filtered_tokens
# keywords

In [40]:
top_keywords = []
# for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
#     print('%8s:\t%.4f' % (word, r))
top_keywords.append(
    sorted(keywords.items(),
           key=lambda x:x[1],
           reverse=True)[:100]
)


keyword_list = []
for keywords in top_keywords:
    words, ranks = zip(*keywords)
    print(words)
    
    keyword_list.append(words)


('개발', '데이터', '보안', '코드', '소프트웨어', '프로', '쿠버네티스', '주니어', '프론트엔드', 'api', 'github', '자바', 'design', '가지', '문제', '도구', '분석', '도커', '서비스', '여기보기', '환경', '클라우드', '코딩', '서버', 'http', '애플리케이션', '사람', '기업', '설계', 'jetpack', '시스템', '기본', '소개', '언어', '공부', '이야기', 'chatgpt', '인터뷰', '사이버', '시큐어', '기능', 'ai', '공격', '데브섹옵스', '작성', 'aws', '페어', '엔진', '이미', '언리얼', 'data', '영향을', '구현', 'java', '운영', '정리', 'game', '테스트', '통신', '머신러닝', '생각', '트리', 'web', '리뷰', '지원', '네트워', '사고', '브라우저', '실제', '경우', '있지만', '성장', '컴퓨터', '이제', '진행', '설명해', '아키텍처', '구축', '앱을', '성능', '매우', '되는', '이유', '싶은', '모놀리식', '파일', 'it', '유연한', '파이', '깃허브', '접근', '신입', '리더', '방식을', '쉬운', '요즘', '일이', 'ott', '글을', '지금')


In [41]:
for k in range(100):
    
    message = '  --  '.join(
        ['%8s (%.3f)' % (top_keywords[0][k][0],top_keywords[0][k][1])]
         )
    print(message)
# 여기까지가 기사 본문으로부터 키워드 추출

      개발 (24.220)
     데이터 (10.803)
      보안 (7.513)
      코드 (6.011)
   소프트웨어 (5.216)
      프로 (5.153)
   쿠버네티스 (4.127)
     주니어 (3.311)
   프론트엔드 (3.301)
     api (3.204)
  github (3.140)
      자바 (3.028)
  design (2.727)
      가지 (2.674)
      문제 (2.650)
      도구 (2.607)
      분석 (2.594)
      도커 (2.562)
     서비스 (2.539)
    여기보기 (2.482)
      환경 (2.477)
    클라우드 (2.453)
      코딩 (2.436)
      서버 (2.351)
    http (2.315)
  애플리케이션 (2.273)
      사람 (2.265)
      기업 (2.245)
      설계 (2.178)
 jetpack (2.123)
     시스템 (2.107)
      기본 (2.101)
      소개 (2.095)
      언어 (2.034)
      공부 (2.032)
     이야기 (2.006)
 chatgpt (2.002)
     인터뷰 (1.974)
     사이버 (1.965)
     시큐어 (1.950)
      기능 (1.926)
      ai (1.897)
      공격 (1.833)
   데브섹옵스 (1.809)
      작성 (1.793)
     aws (1.742)
      페어 (1.715)
      엔진 (1.711)
      이미 (1.709)
     언리얼 (1.692)
    data (1.675)
     영향을 (1.664)
      구현 (1.662)
    java (1.656)
      운영 (1.615)
      정리 (1.607)
    game (1.584)
     테스트 (1.578)
      통신 (1.

 top 100에서 중복되는 키워드들을 제거하고 차이가 있는 키워드만 추출해서 살펴보겠습니다.

In [42]:
keyword_counter = {}

w_l = []
r_l = []
for keywords in top_keywords:
    words, ranks = zip(*keywords)
    for word in words:
        keyword_counter[word] = keyword_counter.get(word, 0) + 1
        w_l.append(word)
    for rank in ranks:
        r_l.append(rank)

csv_list = list(zip(w_l, r_l))
common_keywords = {word for word, count in keyword_counter.items() if count == 3}
len(common_keywords)

0

In [43]:
# csv_list


In [44]:
df_csv = pd.DataFrame(data=csv_list, columns=['words','ranks'])
df_csv = df_csv.sort_values(by=['ranks'], ascending=False)
df_csv

,words,ranks
0,개발,24.219529
1,데이터,10.803485
2,보안,7.512930
3,코드,6.010733
4,소프트웨어,5.215646
...,...,...
95,요즘,1.227968
96,일이,1.227885
97,ott,1.221299
98,글을,1.217100


In [46]:
df_csv.to_csv("../data_crawling_D/keywords/recentit_keywords.csv", mode='w')

## 키워드 분석

## # Reference keywords

In [48]:

# df_techworld
df_techworld_tnt = df_techworld[['news_date', 'total_keywords']]
# df_techworld_tnt

In [ ]:
# 데이터프레임에서 '내용'과 '작성일' 열을 가져옴
import re
from collections import Counter

# 모듈화하기
def get_keyword_monthly_agg(df_techworld_tnt):
    text_data = df_techworld_tnt['total_keywords']
    date_data = df_techworld_tnt['news_date']
    
    
    # 단어 등장 횟수를 저장할 딕셔너리 초기화
    word_counts = {}
    # 월별 단어 집계
    word_n_cnt_list = []
    
    # 데이터프레임을 순회하며 월 별로 단어 등장 횟수 집계
    for text, date in zip(text_data, date_data):
        # 텍스트 토큰화 및 형태소 분석
        
        # 년 추출
        tokens = text.split(",")
        year = date.split('-')[0]
        # 월 추출
        month = date.split('-')[1]
        year_month = year + '-' + month

        # 단어 등장 횟수 집계
        if year_month in word_counts:
            for token in tokens:
                word_counts[year_month][token] = word_counts[year_month].get(token, 0) + 1
        else:
            word_counts[year_month] = {}
            for token in tokens:
                word_counts[year_month][token] = 1
                
    # 횟수가 2이상인 단어만 추출하기
    min_word_count = 2
    top_words_limit = 21 # 최대 20개
    
    # 월 별로 상위 단어 추출
    
    for year_month, counts in word_counts.items():
        # print(f"{month}월")
        # 횟수만 추출하기
        sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        
        # 횟수가 2이상인 단어 필터링
        filtered_counts = [(word, count) for word, count in sorted_counts if count >= min_word_count]
        
        # 만약 2이상인 단어의 수가 5개 미만이면 모든 단어 추출
        if len(filtered_counts) < top_words_limit:
            filtered_counts = sorted_counts
        # 상위 단어 출력
        top_words = filtered_counts[:top_words_limit]
        for word, count in top_words:
            # print(f" {word} | 등장횟수: {count}")
            word_n_cnt_list.append([year_month, word, count])
        # print(word_n_cnt_list)
        print()
    
    return word_n_cnt_list

tw_key_list = get_keyword_monthly_agg(df_techworld_tnt)
# df_tw_key_h_list

In [55]:
# 리스트를 df으로
df_tw_key_list = pd.DataFrame(tw_key_list, columns=['date','words','cnt'])
print('2차원 리스트로 만든 dataframe:\n', df_tw_key_list)

2차원 리스트로 만든 dataframe:
         date words  cnt
0    2023-05  클라우드   34
1    2023-05    출시   20
2    2023-05    기반   18
3    2023-05   글로벌   18
4    2023-05   솔루션   16
..       ...   ...  ...
247  2022-06    테크   26
248  2022-06   데이터   26
249  2022-06   파트너   24
250  2022-06    지원   22
251  2022-06    지능   22

[252 rows x 3 columns]


In [ ]:
# pivot으로 {인덱스 : date, 컬럼: words, 값: cnt}인 테이블 생성
df_tw_key_pivot = df_tw_key_list.pivot(index='date', columns='words', values='cnt')
df_tw_key_pivot.fillna('0', inplace=True) # 결측값 0처리
df_tw_key_pivot = df_tw_key_pivot.apply(pd.to_numeric)
# df_tw_key_pivot

In [ ]:
df_tw_key_pivot.info()

In [68]:
# 각 단어당 cnt 합산을 구한 뒤 상위 100개 리스트에 담기
df_tw_key_pivot_sum = df_tw_key_pivot.sum()
df_tw_key_pivot_sum_srt = df_tw_key_pivot_sum.sort_values(ascending=False)
df_tw_key_pivot_sum_srt

words
클라우드      452.0
출시        362.0
솔루션       349.0
서비스       300.0
사업        263.0
          ...  
공략          6.0
공공          6.0
이스트소프트      6.0
유공          6.0
티맥스소프트      6.0
Length: 65, dtype: float64

In [69]:
# 인덱스에서 word를 빼내고
df_tw_key_pivot_sum_srt = df_tw_key_pivot_sum_srt.reset_index(drop=False)

In [73]:
ref_key_list = df_tw_key_pivot_sum_srt['words'].to_list()
# ref_list

In [74]:
# 100개 reference용 키워드 추출
top_100_ref_list = ref_key_list[:100]    

In [72]:
top_100_kw_list = []
for k in range(100):
    # print(top_keywords[0][k][0])
    top_100_kw_list.append(top_keywords[0][k][0])
print(top_100_kw_list)

['ai', '클라우드', '기업', '서비스', '솔루션', '데이터', '플랫폼', '디지털', '보안', '개발', '글로벌', '출시', '사업', '국내', '전문', '체결', '시장', '지원', '선정', '프로', '구축', '고객', '관리', '시스템', '업무', '발표', '운영', '한국', 'it', '제품', '산업', '메타', '대표', '서울', '비즈니스', '인공', '진행', '분석', '개최', '소프트', '인증', '파트너', '공급', '스마트', '분야', '연구', '로봇', '혁신', '통합', '스타트업', '협력', '자사', '기능', '투자', '올해', '공개', '교육', '환경', '자동', '획득', '인프라', '정보', '기존', '공공', '콘텐츠', '확대', '강화', '온라인', '오픈', '금융', '코리아', '구글', '확장', '영상', '모델', '3d', '추진', 'kt', '참가', 'mo', 'nft', '차세대', '6월', '성장', '가상', '신규', '인텔', '적용', '오라클', '마이', '가능', '가트너', '버추얼', '애플리케이션', 'digital', '의료', '공식', '매출', 'software', 'aws']


## ROUGE-1,2,l 로 성능 평가

In [76]:
from rouge import Rouge

# 모델로 구한 키워드들
rank_list = top_100_kw_list
rank_str = " ".join(rank_list)
# 실제 헤드라인과 내용에서 집계한 키워드들
ref_str = " ".join(top_100_ref_list)
model_out = [rank_str]
# '다양한', '관련', '것이', '새로운', '최근', '분석', '출처:', '활용', '따라', '한다.', '세계', '공동', '가장', '하는', '현재', '기존', '등을', '점유율은', '모든', '다른', '대해', '많은', '순위', '결과를', '또한', '대비', '10', '신규', '이용', '이를', '주요', '모두', '문제', '그러나', '여러', '출시', '이미', '생성', '가능', '이어', '공개', '발표', '이후', '최신', '전했다', '특히', '이상', '인기', '이는', '하지만', '지수', '이러한', '때문에', '핵심', '6월', '아이', '것은', '그리고', '이에', '같이', '동시에', '각각', '참여', '시간', '전체', '하나', '수준', '물론', '다수', '동안', '9월', '어떤', '전망', '혹은', '양자', '전자', '바로', '여전히'
reference = [ref_str]
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

# >>>
# {   'rouge-1': {   'f': 0.6279006234427593,
#                    'p': 0.8604497354497355,
#                    'r': 0.5273531655225019},
#     'rouge-2': {   'f': 0.3883256484545606,
#                    'p': 0.5244559362206421,
#                    'r': 0.32954545454545453},
#     'rouge-l': {   'f': 0.6282785202429159,
#                    'p': 0.8122895622895623,
#                    'r': 0.5369305616983636}}

{'rouge-1': {'r': 0.6307692307692307, 'p': 0.41, 'f': 0.4969696921946741},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.2153846153846154, 'p': 0.14, 'f': 0.16969696492194689}}